In [1]:
############################################################################################################################
## Script: AGOLSQLSync.py
## Author: Jeremy Mullins, Derek Morgan
## Date:
##
## Description:
##        This script is the test script for syncing data between ArcGIS Online
##        and an MS SQL Server database.
##
## Required prerequisites:
##        - ArcGIS API for Python
##            (https://developers.arcgis.com/python/)
##
##        - pyodbc Module
##            (https://github.com/mkleehammer/pyodbc)
##                to install after download, open Python command
##                prompt and type the following:
##                    -- conda install pyodbc  --
##
##        - MS ODBC Driver for SQL Server (v17)
##            (https://docs.microsoft.com/en-us/sql/connect/odbc/download-odbc-driver-for-sql-server?view=sql-server-2017)
##
############################################################################################################################

In [2]:
# import all necessary modules
from arcgis.gis import GIS
import pyodbc as sql
import pandas as pd
import configparser
import zipfile
import arcpy
import sys
import os

In [3]:
#read config file
config = configparser.ConfigParser()
config.read('config_jm.ini')

# assign config variables
agolURL = config['AGOL']['URL']
agolUSER = config['AGOL']['USER']
agolPW = config['AGOL']['PW']
sqlDRVR = config['SQL']['SQLDRVR']
sqlSERV = config['SQL']['SERVER']
sqlDB = config['SQL']['DB']
sqlUSER = config['SQL']['USER']
sqlPW = config['SQL']['PW']
iD = config['SCRIPT']['itemID']
csvNAME = config['SCRIPT']['csvTITLE']
csvLOC = config['SCRIPT']['csvLOC']
zipLOC = config['SCRIPT']['zipLOC']
csvDOC = config['SCRIPT']['csvDOC']
newTEMP1 = config['SCRIPT']['newTEMP1']
newTEMP2 = config['SCRIPT']['newTEMP2']
csv2TEMP = config['SCRIPT']['csv2TEMP']
newINS1 = config['SCRIPT']['newINS1']
newINS2 = config['SCRIPT']['newINS2']
SQL2CSV = config['SCRIPT']['SQL2CSV']
sql2CSVout = config['SCRIPT']['sql2CSVout']
delTEMP1 = config['SCRIPT']['delTEMP1']
delTEMP2 = config['SCRIPT']['delTEMP2']

In [4]:
#Sign into AGOL acct
gis = GIS(agolURL,agolUSER,agolPW)

#Get feature layer in question
featureLayer = gis.content.get(iD)

In [5]:
#Export feature layer as CSV (ZIP file)
output_file = featureLayer.export(title=csvNAME,export_format="CSV")
output_file.download(csvLOC)

#Unzip downloaded ZIP folder containing feature layer as CSV
zip_ref = zipfile.ZipFile(zipLOC,'r')
zip_ref.extractall(csvLOC)
zip_ref.close()

#Delete ZIP folder on disk and CSV collection file in AGOL
os.remove(zipLOC)
output_file.delete()

#Read CSV using panda; replace NaNs with '000'
df = pd.read_csv(csvDOC)
df.fillna('000',inplace=True)

In [6]:
#Connect to SQL db and assign cursor
conn = sql.connect('Driver='+sqlDRVR+';'
                      'Server='+sqlSERV+';'
                      'Database='+sqlDB+';'
                      'trusted_connection=yes;'
                      'UID='+sqlUSER+';'
                      'PWD='+sqlPW+';')
cursor = conn.cursor()

#Create temporary tables in SQL db
cursor.execute(newTEMP1)
cursor.commit()

cursor.execute(newTEMP2)
cursor.commit()

#Insert CSV into temporary table
for index,row in df.iterrows():
    cursor.execute(csv2TEMP,row['OBJECTID'],
                   row['Building Identifier'],
                   row['Description'],
                   row['Classrooms'],
                   row['TypeCode'],
                   row['GlobalID'])
cursor.commit()

#Compare tables and look for non-duplicated GlobalIDs
cursor.execute(newINS1)
cursor.commit()

In [7]:
#Compare tables and look for any updated records in prod table
cursor.execute(newINS2)
cursor.commit()

#Export SQL data to CSV
outCSVscript = SQL2CSV
df2 = pd.read_sql_query(outCSVscript,conn)
df2.to_csv(sql2CSVout)

#https://stackoverflow.com/questions/
#     18977387/how-to-export-sql-server-result-to-excel-in-python
#https://datatofish.com/export-dataframe-to-csv/

In [8]:
#Delete temporary table
cursor.execute(delTEMP1)
cursor.commit()

cursor.execute(delTEMP2)
cursor.commit()

In [9]:
#Close and delete cursor; close SQL db connection
cursor.close()
del cursor
conn.close()

#Remove AGOL CSV
os.remove(csvDOC)